In [1]:
import pandas as pd

# Load the dataset
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')


In [2]:
df_cap.head()

,Facility,Max_Capacity
0,0,15060
1,1,15491
2,2,18463
3,3,15757
4,4,18780


In [3]:
df_de.head()

,Region,Demand
0,0,5157
1,1,2449
2,2,6580
3,3,5375
4,4,2727


In [4]:
df_cost.head()

,Facility,Fixed_Cost,Revenue_Region_1,Revenue_Region_2,Revenue_Region_3,Revenue_Region_4,Revenue_Region_5,Revenue_Region_6,Revenue_Region_7,Revenue_Region_8,...,Revenue_Region_22,Revenue_Region_23,Revenue_Region_24,Revenue_Region_25,Revenue_Region_26,Revenue_Region_27,Revenue_Region_28,Revenue_Region_29,Revenue_Region_30,Revenue_Region_31
0,0,30977216,168,165,370,289,139,275,212,492,...,320,473,422,353,381,76,450,64,70,462
1,1,19988490,214,302,221,314,374,195,210,187,...,81,305,94,469,270,299,84,442,71,390
2,2,46375256,235,167,275,294,377,303,107,422,...,119,402,469,118,199,364,101,448,458,456
3,3,33623823,291,390,464,487,419,336,56,284,...,398,398,79,338,443,57,94,490,265,346
4,4,45544254,186,150,209,307,263,371,348,207,...,429,101,272,398,237,222,214,52,326,428


## Question 1

### (a) What condition(s) must be met for the optimal objective function value to be positive?

#### For the optimal objective function value to be positive, the total revenue from distributing brakes must exceed the total fixed cost of establishing the distribution centers. This requires that at least one DC is opened and that the volume of brakes shipped from the open facility or facilities—when multiplied by the corresponding per-unit revenues—is high enough to cover the fixed cost of operating those centers. Essentially, there must exist a feasible solution where the revenue generated from the shipments outweighs the fixed investment required to set up the network, ensuring a net profit greater than zero.

### (b) Interpret the constraint that links the decision to open a distribution center with their distribution plan, given that the model addresses a capacity planning problem over the next 10 years.

#### The linking constraint, formulated as x[i, j] ≤ 0.45 * Demand[j] * y[i], serves a dual purpose. First, it ensures that if a distribution center is not opened (i.e., y[i] = 0), then no brakes can be distributed from that facility (forcing x[i, j] to zero for all regions j). Second, it limits the volume that can be shipped from an open facility to any region to at most 45% of that region’s demand. This linkage is critical in a 10-year capacity planning problem because it ties the irreversible investment decision of opening a facility to its operational use. Essentially, it guarantees that only facilities that are built can contribute to the distribution plan, and even then, they must operate within realistic bounds that reflect regional demand constraints.

### (c) Benchmarking is useful for providing context. Therefore using Gurobi and assuming constraints 1−8 above are omitted, what would the optimal profit be?

In [2]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data files
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists for facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, and fixed costs
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build a revenue dictionary: revenue[(i,j)] = revenue from facility i to region j.
# Revenue columns in df_cost are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'.
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# Create a new Gurobi model for the benchmark problem
model = Model("BenchmarkModel")
model.setParam('OutputFlag', 1)  # Enable solver output

# Decision Variables:
# y[i] is a binary variable indicating whether facility i is opened.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] is a continuous variable representing the number of brakes shipped from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# Basic Constraints (Benchmark Model):
# 1. Capacity Constraint: The total shipments from facility i cannot exceed its capacity multiplied by its open decision.
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# 2. Demand Constraint: The total shipments to region j cannot exceed the region's demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# Objective Function:
# Maximize total revenue from shipments minus the fixed cost of opening facilities.
model.setObjective(
    quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# Optimize the model
model.optimize()

# Print the optimal objective value (benchmark profit)
print("Optimal Profit (Benchmark):", model.objVal)


Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 544 columns and 1071 nonzeros
Model fingerprint: 0x6d10af01
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [5e+01, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 48 rows, 544 columns, 1071 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best object

### (d) After solving the full MILP model with Gurobi, what is the optimal profit?

### (e) Discuss the value of benchmarking by comparing the optimal objective function values from the previous two questions. What managerial insights can be drawn from this comparison?

### (f) If we solved the linear relaxation of this problem, it would be classified as a linear program. Although this would simplify the problem, the resulting optimal solution would not be integral. Explain why using this method to solve the problem faster is not practical in this case.

### (g) For the full MILP model, how many feasible solutions are within 1% of the optimal solution? What is the smallest objective function value of the feasible solution(s) you have found?

### (h) Finding both the optimal solution and several sub-optimal solutions within a relative optimality gap can be challenging for more complex problems. How can you configure Gurobi to return the first feasible solution it finds within a specified percentage of the optimal solution? Using this parameter setting, what is the objective function value for a solution within 5% of optimality?

### (i) Write down two additional Gurobi parameters that can be used to find feasible but sub-optimal solutions for difficult optimization problems, and briefly explain why they are applicable.

### (j) Explain why, in cases like this, using Gurobi parameters to solve an approximate version of an optimization problem is preferable to solving an LP relaxation of the same model.